In [ ]:
from pathlib import Path

from gettsim import main
from ttsim import MainTarget

GETTSIM_ROOT = Path.cwd().parent.parent / "src" / "_gettsim"

# Modifying Taxes and Transfers

GETTSIM's design allows you to go beyond the depiction of the current (or historical)
tax and transfer system. Analyzing counterfactual reform scenarios, ranging from small
changes of certain parameters of the tax and transfer system, to the introduction of
large-scale reforms, is a common use case.

This tutorial showcases how to modify taxes and transfers when using GETTSIM.

Here, we focus on small reforms to the means-tested social welfare benefits for
the unemployed (German: Bürgergeld; until 2022: Arbeitslosengeld II). We pick this
example because Bürgergeld is a fairly complex system that uses the entire range of
objects TTSIM offers.

## Status Quo

The heart of GETTSIM's tax and transfer implementation is the `policy_date`-specific
`policy_environment`. The `policy_environment` holds all the parameters and functions
necessary to compute the taxes and transfers for a given policy date. The
`policy_environment` is a nested dictionary with strings as keys and `TTSIM-objects`
as values.

The `policy_environment` has 3 broad types of objects (with many subclasses):
- Objects that operate on columns of data, either provided by the user or computed by
  previous functions. We call those `ColumnObjects`.
- Objects that contain parameters. We call those `ParamObject`s.
- Objects that process parameters in a way such that they can be used by the
  `ColumnObjects`. We call those `ParamFunction`s.

The first step when modifying taxes and transfers is to create the current policy
environment.

In [ ]:
status_quo_environment = main(
    main_target=MainTarget.policy_environment,
    policy_date_str="2025-01-01",
    backend="numpy",
)

In [ ]:
import numpy as np

main(
    main_target=MainTarget.templates.input_data_dtypes,
    policy_date_str="2025-01-01",
    input_data={
        "tree": {
            "p_id": np.array([1]),
            "arbeitslosengeld_2": {
                "bruttoeinkommen_m": np.array([0.0]),
            },
        }
    },
    tt_targets={"tree": {"arbeitslosengeld_2": {"betrag_m_bg": None}}},
)

### UThe 

## Replacing/Adding Parameters

## Replacing/Adding Functions

## 